In [ ]:
# import sys
# 
# from tests.smoke_tests.olesik_et_al_2022.test_simulation import settings
# 
# if 'google.colab' in sys.modules:
#     !pip --quiet install open-atmos-jupyter-utils
#     from open_atmos_jupyter_utils import pip_install_on_colab
#     pip_install_on_colab('PyMPDATA-examples')

In [ ]:
import matplotlib.pyplot as plt

import os

import pandas

# os.environ["NUMBA_DISABLE_JIT"] = "1"

from asian_option import AsianArithmetic, Settings
import numpy as np
from matplotlib import pyplot
import pandas as pd
from ipywidgets import IntProgress
# from open_atmos_jupyter_utils import show_plot, show_anim
from PyMPDATA_examples.Magnuszewski_et_al_2025.common import OPTIONS
from monte_carlo import BSModel, FixedStrikeGeometricAsianOption, FixedStrikeArithmeticAsianOption
import time
from tqdm import tqdm
import PyMPDATA_examples.utils.financial_formulae.asian_option as asian_analytic
from PyMPDATA_examples.Magnuszewski_et_al_2025 import barraquand_data

pandas.options.display.float_format = '{:,.3f}'.format

In [ ]:
s_min = 50
s_max = 200
mc_n_paths = [1000, 10000]
mc_seed = 42
mc_path_points = 1000
spot = 100
risk_free_rate=0.1


In [ ]:
def run_numeric_and_mc(params, *, nx, ny, nt, variant):
    settings = Settings(**params, r=risk_free_rate, S_max=s_max, S_min=s_min)
    mc_model = BSModel(T=params['T'],
                       sigma=params['sgma'],
                       r=risk_free_rate,
                       M=mc_path_points,
                       S0=spot,
                       seed=mc_seed)
    simulations = {k:AsianArithmetic(settings, nx=nx, ny=ny, nt=nt, variant=variant, options=OPTIONS[k]) for k in OPTIONS}
    results = {}
    
    for k, simulation in simulations.items():
        simulation.step(simulation.nt)
        simulation_price = simulation.solver.advectee.get()[:, 0]
        results[k] = np.interp(spot, simulation.S, simulation_price)
    for mc_n_path in mc_n_paths:
        arithmetic_option = FixedStrikeArithmeticAsianOption(params['T'], params['K'], variant, mc_model, mc_n_path)
        results[f"MC_{mc_n_path}"] = arithmetic_option.price_by_mc()
    return results

In [154]:
discretization_parameters = {
    (.1,.25,95):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.25,100):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.25,105):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.5,95):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,.5,100):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,.5,105):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,1,95):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.1,1,100):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.1,1,105):{'nx': 251, 'ny': 250, 'nt': 1500},
    
    (.2,.25,95):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.25,100):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.25,105):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.5,95):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,.5,100):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,.5,105):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,1,95):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.2,1,100):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.2,1,105):{'nx': 251, 'ny': 250, 'nt': 1500},
}

In [ ]:
barraquand_df = pd.DataFrame(columns=barraquand_data.headers)
for line in barraquand_data.table.strip('\n').split('\n'):
    data_row = line.split(',')
    if len(data_row) > 0:
        barraquand_df.loc[len(barraquand_df)] = data_row
barraquand_df['call_price'] = barraquand_df['call_price'].astype(float)
barraquand_df['put_price'] = barraquand_df['put_price'].astype(float)

In [ ]:
def calculate_row(row_idx):
    row_data = barraquand_df.iloc[row_idx].astype(float)
    nx,ny,nt = discretization_parameters[(row_data['sigma'], row_data['T'], row_data['K'])].values()
    simulation_params = {
        'sgma':row_data['sigma'],
        'T':row_data['T'],
        'K':row_data['K']
    }
    call_price = row_data['call_price']
    put_price = row_data['put_price']
    results_call = run_numeric_and_mc(simulation_params, nx=nx, ny=ny, nt=nt, variant='call')
    results_put = run_numeric_and_mc(simulation_params, nx=nx, ny=ny, nt=nt, variant='put')
    return {k: round(v,3) for k, v in results_call.items()}, {k: round(v,3) for k, v in results_put.items()}, simulation_params, call_price, put_price

In [128]:
results_header = ['sigma', 'T', 'K', 'BP_call', 'UPWIND_call', 'MPDATA_call', 'MC_1000_call', 'MC_10000_call', 'BP_put', 'UPWIND_put', 'MPDATA_put', 'MC_1000_put', 'MC_10000_put']
results_df = pd.DataFrame(columns=results_header)
for i in tqdm(range(12)):
    call, put, params, call_bp, put_bp = calculate_row(i)
    new_row = [*params.values(), call_bp, *call.values(), put_bp, *put.values()]
    # print(new_row)
    results_df.loc[len(results_df)] = new_row
results_df['K'] = results_df['K'].astype(int)

100%|██████████| 18/18 [04:46<00:00, 15.92s/it]


In [129]:
display(results_df)

,sigma,T,K,BP_call,UPWIND_call,MPDATA_call,MC_1000_call,MC_10000_call,BP_put,UPWIND_put,MPDATA_put,MC_1000_put,MC_10000_put
0,0.100,0.250,95,6.132,6.861,6.051,6.031,6.116,0.013,0.737,0.012,0.022,0.010
1,0.100,0.250,100,1.869,3.496,1.853,1.835,1.852,0.626,2.244,0.626,0.702,0.623
2,0.100,0.250,105,0.151,1.403,0.238,0.142,0.148,3.785,5.032,3.794,3.886,3.796
3,0.100,0.500,95,7.248,8.109,7.142,7.106,7.215,0.046,0.904,0.041,0.067,0.041
4,0.100,0.500,100,3.100,4.750,3.046,3.026,3.071,0.655,2.298,0.650,0.743,0.653
5,0.100,0.500,105,0.727,2.395,0.820,0.710,0.718,3.039,4.700,3.058,3.183,3.056
6,0.100,1.000,95,9.313,10.080,9.240,9.138,9.280,0.084,0.837,0.077,0.120,0.078
7,0.100,1.000,100,5.279,6.661,5.222,5.170,5.253,0.577,1.938,0.568,0.676,0.576
8,0.100,1.000,105,2.313,3.999,2.346,2.299,2.294,2.137,3.798,2.139,2.330,2.140
9,0.200,0.250,95,6.500,7.742,6.418,6.369,6.474,0.379,1.624,0.368,0.452,0.369


In [153]:
call, put, _, call_bp , put_bp = calculate_row(15)
print(f"{call=}{call_bp}\n{put=}{put_bp}")

call={'UPWIND': 11.231, 'MPDATA': 10.356, 'MC_1000': 10.127, 'MC_10000': 10.291}10.336
put={'UPWIND': 1.909, 'MPDATA': nan, 'MC_1000': 1.271, 'MC_10000': 1.093}1.099


In [146]:
call, put, _, call_bp , put_bp = calculate_row(13)
print(f"{call=}{call_bp}\n{put=}{put_bp}")

KeyboardInterrupt: 

In [143]:

call, put, _, call_bp , put_bp = calculate_row(14)
print(f"{call=}{call_bp}\n{put=}{put_bp}")

AssertionError: Lambda squared should be more than 2 for stability 1.5252234533362996

## sigma = 0.2

### T = 0.25

### T = 0.5

In [ ]:
params = {
        'T': 0.5,
        'K': 95,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 7.793,
    }
run_numeric_and_mc(params, nx=101, ny=300, nt=3000)

In [ ]:
params = {
        'T': 0.5,
        'K': 100,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 4.548,
    }
run_numeric_and_mc(params, nx=101, ny=300, nt=3000)

In [ ]:
params = {
        'T': 0.5,
        'K': 105,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 2.241,
    }
run_numeric_and_mc(params, nx=101, ny=400, nt=3000)

### T = 1

In [ ]:
params = {
        'T': 1,
        'K': 95,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 10.336,
    }
run_numeric_and_mc(params, nx=101, ny=300, nt=3000)

In [ ]:
params = {
        'T': 1,
        'K': 100,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 7.079,
    }
run_numeric_and_mc(params, nx=121, ny=200, nt=4000)

In [ ]:
params = {
        'T': 1,
        'K': 105,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 4.539,
    }
run_numeric_and_mc(params, nx=101, ny=250, nt=5000)

## sigma = 0.4

### T = 0.25

In [ ]:
params = {
        'T': 0.25,
        'K': 95,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 8.151,
    }
run_numeric_and_mc(params, nx=101, ny=300, nt=3000)

In [ ]:
params = {
        'T': 0.25,
        'K': 100,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 5.218,
    }
run_numeric_and_mc(params, nx=101, ny=200, nt=3000)

In [ ]:
params = {
        'T': 0.25,
        'K': 105,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 3.106,
    }
run_numeric_and_mc(params, nx=81, ny=200, nt=5000)

### T = 0.5

In [ ]:
params = {
        'T': 0.5,
        'K': 95,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 10.425,
    }
run_numeric_and_mc(params, nx=81, ny=200, nt=5000)

In [ ]:
params = {
        'T': 0.5,
        'K': 100,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 7.650,
    }
run_numeric_and_mc(params, nx=81, ny=200, nt=5000)

In [ ]:
params = {
        'T': 0.5,
        'K': 105,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 5.444,
    }
run_numeric_and_mc(params, nx=81, ny=150, nt=5000)

### T = 1

In [ ]:
params = {
        'T': 1,
        'K': 95,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 13.825,
    }
run_numeric_and_mc(params, nx=101, ny=102, nt=4000)

In [ ]:
params = {
        'T': 1,
        'K': 100,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 11.213,
    }
run_numeric_and_mc(params, nx=101, ny=102, nt=4000)

In [ ]:
params = {
        'T': 1,
        'K': 105,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 8.989,
    }
run_numeric_and_mc(params, nx=101, ny=102, nt=4000)

# Put

## sigma = 0.1

### T = 0.25

In [ ]:
qparams = {
        'T': 0.5,
        'K': 95,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 0.731,
    }
run_numeric_and_mc(params, nx=200, ny=200, nt=1000, variant='put')

In [ ]:
params = {
        'T': 0.5,
        'K': 100,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 2.102,
    }
run_numeric_and_mc(params, nx=200, ny=200, nt=1000, variant='put')

In [ ]:
params = {
        'T': 0.5,
        'K': 105,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 4.552,
    }
run_numeric_and_mc(params, nx=200, ny=200, nt=1000, variant='put')

In [ ]:
params = {
        'T': 1,
        'K': 95,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 1.099,
    }
run_numeric_and_mc(params, nx=200, ny=200, nt=4000, variant='put')

In [ ]:
params = {
        'T': 1,
        'K': 100,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 2.369,
    }
run_numeric_and_mc(params, nx=200, ny=200, nt=4000, variant='put')

In [ ]:
params = {
        'T': 1,
        'K': 105,
        'r': 0.1,
        'sgma': 0.2,
        'spot': 100,
        'true_price': 4.356,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2000, variant='put')

In [ ]:
params = {
        'T': 0.25,
        'K': 95,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 2.025,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2000, variant='put')

In [ ]:
params = {
        'T': 0.25,
        'K': 100,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 3.970,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2000, variant='put')

In [ ]:
params = {
        'T': 0.25,
        'K': 105,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 6.735,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2000, variant='put')

In [ ]:
params = {
        'T': 0.5,
        'K': 95,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 3.215,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2000, variant='put')

In [ ]:
params = {
        'T': 0.5,
        'K': 100,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 5.197,
    }

run_numeric_and_mc(params, nx=100, ny=100, nt=2000, variant='put')

In [ ]:
params = {
        'T': 0.5,
        'K': 105,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 7.748,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2000, variant='put')

In [ ]:
params = {
        'T': 1,
        'K': 95,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 4.550,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2420, variant='put')

In [ ]:
params = {
        'T': 1,
        'K': 100,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 6.465,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2300, variant='put')

In [ ]:
params = {
        'T': 1,
        'K': 105,
        'r': 0.1,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 8.767,
    }
run_numeric_and_mc(params, nx=100, ny=100, nt=2600, variant='put')

In [ ]:
def plot_solution(
    settings,
    frame_index,
    ax,
    history,
    S_linspace,
    arithmetic_by_mc,
    arithmetic_by_mc_std,
    option_type: str,
    variant,
    A_space,
    S_space
):
    params = {
        k: v for k, v in settings.params.__dict__.items() if not k.startswith("K")
    }
    if variant == "call":
        BS_price_func = Black_Scholes_1973.c_euro
        Amer_price_func = Bjerksund_and_Stensland_1993.c_amer
        geometric_price_func = asian_analytic.geometric_asian_average_price_c
    else:
        BS_price_func = Black_Scholes_1973.p_euro
        Amer_price_func = Bjerksund_and_Stensland_1993.p_amer
        geometric_price_func = asian_analytic.geometric_asian_average_price_p
    range_lower, range_upper = 80, 120
    s_indexes_within_range = [i for i, v in enumerate(S_linspace) if range_lower<=v<=range_upper]
    # s_indexes_within_range = [i for i, v in enumerate(S_linspace)]
    ax.plot(
        S_linspace[s_indexes_within_range],
        (
            BS_price_func(
                S=S_linspace[s_indexes_within_range], K=settings.params.K, **params, b=settings.params.r
            )
        ),
        label="European analytic (Black-Scholes '73)",
        linestyle="--",
        alpha=0.5,
        c="black"
    )
    # ax.plot(
    #     S_linspace,
    #     (
    #         Amer_price_func(
    #             S=S_linspace, K=settings.params.K, **params, b=settings.params.r
    #         )
    #     ),
    #     label="American analytic (Bjerksund & Stensland '93)", linestyle='--'
    # )
    
    # print(settings.rh[:-1].shape)
    # print(history[frame_index][0,:].shape)
    ax.bar(
        S_space[:-1][s_indexes_within_range],
        history[frame_index][:, 0][s_indexes_within_range],
        width=S_space[1:][s_indexes_within_range] - S_space[:-1][s_indexes_within_range],
        # label=f"MPDATA solution ({option_type})",
        alpha=0.3,
        align="edge",
        color="orange"
    )
    # ax.bar(
    #     settings.rh[:-1],
    #     history[0][0,:],
    #     width=settings.rh[1:] - settings.rh[:-1],
    #     label=f"Terminal condition (discounted payoff)",
    #     alpha=0.5,
    #     align="edge"
    # )
    ax.plot(
        S_linspace[s_indexes_within_range],
        history[frame_index][:, 0][s_indexes_within_range],
        label=f"Asian arithmetic MPDATA solution",
        marker=".",
        # alpha=0.7,
        c="orange"
    )
    
    ax.plot(
        S_linspace[s_indexes_within_range],
        arithmetic_by_mc[s_indexes_within_range],
        # arithmetic_by_mc_std[s_indexes_within_range],
        label="Asian arithmetic by Monte-Carlo",
        # linestyle=":",
        marker="+",
        # alpha=0.5,
        c="blue"
    )
    ax.plot(
        S_linspace[s_indexes_within_range],
        (
            geometric_price_func(
                S=S_linspace[s_indexes_within_range], K=settings.params.K, **params, dividend_yield=0
            )
        ),
        label="Asian geometric analytic (Kemna & Vorst 1990)",
        # alpha=0.5,
        linestyle="--",
        c="green"
    )
    # ax.plot(
    #     A_space,
    #     history[0][0, :],
    #     label=f"Terminal condition (discounted payoff)",
    #     marker=".",
    #     alpha=0.5,
    # )
    ax2 = ax.twinx()
    # on ax2 plot discretized ticks x/Δx
    ax2.set_xticks(np.arange(0, len(settings.rh), 5))
    ax.legend(loc="upper left")
    ax.grid()
    minmax = (np.amin(history[0]), np.amax(history[0]))
    span = minmax[1] - minmax[0]
    # ax.set_ylim(minmax[0] - 0.05 * span, minmax[1] + 0.25 * span)
    # ax.set_title(f"Asian option valuation for: {settings.params.__dict__}")
    # ax.set_xlabel("underlying S(t=0)=A(t=0) (and A(T) for terminal condition)")
    # ax.set_xlabel("average at t=T for payoff, spot at t=0 for other datasets")
    ax.set_xlabel("spot price at t=0 ($)")
    ax.set_ylabel("instrument value ($)")

In [ ]:
params = {
        'T': 1,
        'K': 100,
        'r': 0.08,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 8.767,
    }
run_numeric_and_mc(params, nx=21, ny=31, nt=500, variant='call')

In [ ]:
params = {
        'T': 1,
        'K': 100,
        'r': 0.08,
        'sgma': 0.4,
        'spot': 100,
        'true_price': 8.767,
    }
run_numeric_and_mc(params, nx=21, ny=31, nt=500, variant='call')